In [ ]:
#!pip install langchain openai langchain-openai google-cloud-secret-manager scipy datasets

from transformers import pipeline
import scipy
from langchain import PromptTemplate, LLMChain
from langchain_openai import OpenAI
import torch
from IPython.display import Image
from datasets import load_dataset
# Primero autenticamos el usuario :
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

from google.cloud import secretmanager

# Creamos un Cliente de SecretManager:
client = secretmanager.SecretManagerServiceClient()
secret_name = "openai-token"
project_id = '196177837977'

# Contruimos a F-String:
resource_name = f"projects/{project_id}/secrets/{secret_name}/versions/latest"

# Obtenemos el secreto :
response = client.access_secret_version(request={"name": resource_name})
secret_string = response.payload.data.decode('UTF-8')

llm = OpenAI(openai_api_key=secret_string, temperature=1)

captioner = pipeline("image-to-text",model="Salesforce/blip-image-captioning-base", max_new_tokens=20)


def image2text(image):
    text_result = captioner(image)
    return text_result[0]['generated_text']

def crear_historia(topic):
   template = """
   You are a writer and story teller.
   Your task is generate short stories from a shrot description. The story cannot have more than 50 words.
   CONTEXT: {topic}
   STORY:
   """
   topic_template = PromptTemplate(template=template, input_variables=['topic'])

   topic_chain = LLMChain(llm=llm, prompt=topic_template)
   response = topic_chain.invoke(topic)
   print(response.get('text'))
   return response.get('text')

def crear_audio(text):
  synthesiser = pipeline("text-to-speech", model="microsoft/speecht5_tts")
  embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
  speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

  speech = synthesiser(f"{text}", forward_params={"speaker_embeddings": speaker_embedding})

  scipy.io.wavfile.write("historia.wav", rate=speech["sampling_rate"], data=speech["audio"])


generated_topic = image2text("https://huggingface.co/datasets/Narsil/image_dummy/resolve/main/parrots.png")

historia = crear_historia(generated_topic)
crear_audio(historia)


The birds sat next to each other on the branch, enjoying the warm spring breeze. One was a vibrant blue with a melodic voice, while the other was a soft yellow with a playful spirit. They were an odd pair, but their differences brought them closer together. Together, they sang the most beautiful duet.
